# Visualizing ADCIRC data

[Datashader](http://datashader.org)'s support for irregular triangular meshes allows datasets from such simulations to be rendered onscreen efficiently. This notebook shows an example of rendering a ADCIRC tidal database of the US Eastern coast, Once Datashader and the dataset are installed, you can run this notebook yourself to get a live version with interactive zooming for the plots that support it.

Here you can enable the wheel-zoom tool in the toolbar, then use scrolling and your pointer to zoom and pan in the plot. As always with datashader, the data is provided to the browser in only one resolution to start with, and it will be updated when you zoom in

In [ ]:
from gridgeo.ugrid import ugrid
from netCDF4 import Dataset


url = 'http://gamone.whoi.edu/thredds/dodsC/usgs/vault0/models/tides/ec2015/f53.ncml'

with Dataset(url) as nc:
    grid = ugrid(nc)
    z = nc['Amp'][0, :]

In [ ]:
import numpy as np


data = np.vstack(
    (grid['nodes']['x'], grid['nodes']['y'], z)
).T

In [ ]:
import pandas as pd


verts = pd.DataFrame(data, columns=['x','y','z'])
tris = pd.DataFrame(grid['faces'], columns=['v0', 'v1', 'v2'])

In [ ]:
import datashader


%time mesh = datashader.utils.mesh(verts, tris)

In [ ]:
cvs = datashader.Canvas(plot_height=500, plot_width=500)

%time agg = cvs.trimesh(verts, tris, mesh=mesh)

In [ ]:
import holoviews


holoviews.extension('bokeh')

In [ ]:
import geoviews
from holoviews.operation.datashader import datashade
import palettable


cmap = palettable.cmocean.sequential.Deep_20.mpl_colormap

%opts WMTS [width=500 height=500]

tiles = geoviews.WMTS('https://maps.wikimedia.org/osm-intl/{Z}/{X}/{Y}@2x.png')

%time points = geoviews.operation.project_points(geoviews.Points(verts, vdims=['z']))

tiles * datashade(
    holoviews.TriMesh((tris, points)),
    cmap=cmap,
    normalization='linear',
    aggregator=datashader.mean('z'),
    precompute=True
)

Here you can enable the wheel-zoom tool in the toolbar, then use scrolling and your pointer to zoom and pan in the plot.  As always with datashader, the data is provided to the browser in only one resolution to start with, and it will be updated when you zoom in only if you have a running Python process, and are not just viewing this on a static web page.